In [1]:
import yaml
import data
from data.data_split import kfsplit
import numpy as np

In [2]:
import pandas as pd 
import os
import torch

from torch.cuda.amp import autocast as autocast, GradScaler
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data.distributed import DistributedSampler
import gc 
gc.enable()
from data.datasets import prepare_loaders

import wandb
from model.head import OriginModel,LR,get_model

import torch
import os
import gc

from model.trainer import train,predict
from utils.wandb_init import get_run,wandb_utils
from utils.train import get_score,train_set
from sklearn.metrics import precision_score,recall_score,f1_score
from data.read_data import select_data,select_data,TrainData,file_split,DataByFile,DataByFileHard
from data.data_split import kfsplit
from data.embedding_init import get_tokenizer,get_embedding
from data.datasets import DatasetRetriever
import nltk
from collections import defaultdict
from torchtext.data.utils import ngrams_iterator
import json
from tqdm import tqdm

2022-11-02 22:39:11.733609: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
yaml_path = "./config/train.yaml"
CONFIG,Logger,DEVICE = train_set(yaml_path,experimentName = None,upload = False,filename = "./test/logs",is_notebook = True)

CONFIG["run_db"] = False
CONFIG["num_hidden_layers"] = 12
CONFIG["DATA_NUM_RATE"] = 0.2
CONFIG["EPOCHS"] = 35
CONFIG["DEBUG_MODEL"] = False

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adreambear. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
with open(os.path.join(CONFIG["INPUT_DIR"],"Schema"),"r") as f:
    line = f.readlines()[0].strip()
    column_name = line.split("\t")
    CONFIG["Logger"].info(f"column_name :{column_name}")
    CONFIG["COLUMN_NAME"] = column_name
head = CONFIG["COLUMN_NAME"]
input_dir = CONFIG["INPUT_DIR"]


column_name :['Query', 'keyword', 'query_len', 'emotion_value', 'label']


In [5]:
from data.read_data import read_data_file

positive_train_file = os.path.join(input_dir,"PositiveTrain")
positive_test_file = os.path.join(input_dir,"PositiveTest")

positive_train_df = read_data_file(positive_train_file,False,5000,head = head)
positive_test_df = read_data_file(positive_test_file,False,5000,head = head)
positive_train_df["label"],positive_test_df["label"] = positive_train_df["label"].astype(int),positive_test_df["label"].astype(int)

# read negative data
negative_train_file = os.path.join(input_dir,"NegativeTrain")
negative_test_file = os.path.join(input_dir,"NegativeTest")

negative_train_df = read_data_file(negative_train_file,False,5000,head = head)
negative_test_df = read_data_file(negative_test_file,False,5000,head = head)
negative_train_df["label"],negative_test_df["label"] = negative_train_df["label"].astype(int),negative_test_df["label"].astype(int)

# train_df = pd.concat([positive_train_df,negative_train_df],axis = 0)
# test_df = pd.concat([positive_test_df,negative_test_df],axis = 0)

# all_text = pd.concat([train_df,test_df],axis = 0)["Query"]

In [6]:
negative_train_df["Query"]=negative_train_df.apply(lambda row:row[0].replace(row[1],"[MASK]"),axis=1)
negative_keyword_counts = negative_train_df["keyword"].value_counts().to_dict()


ValueError: Columns must be same length as key

In [7]:
appear_file="/root/autodl-tmp/epsave/data/raw_data/target_keywords_drug.txt"
appearword=[]
with open(appear_file,"r",encoding="UTF-8") as file_obj:
    lines=file_obj.readlines()
    for line in lines:
        appearword.append(line.strip())

keyword_file="/root/autodl-tmp/epsave/data/raw_data/euphemism_answer_drug.txt"

keyword_dict=defaultdict(list)
with open(keyword_file,"r",encoding="UTF-8") as file_obj:
    for line in file_obj:
        ans = line.split(':')[0].strip().lower()
        for i in line.split(':')[1].split(';'):
            keyword_dict[ans].append(i.strip().lower())

keyword_file = "/root/autodl-tmp/epsave/data/raw_data/euphemism_answer_drug.txt"
euphemism_answer = defaultdict(list)
with open(keyword_file, 'r') as fin:
    for line in fin:
        ans = line.split(':')[0].strip().lower()
        for i in line.split(':')[1].split(';'):
            euphemism_answer[i.strip().lower()].append(ans)

In [8]:
from collections import defaultdict
keyword_count = defaultdict(int)
for key,word_list in keyword_dict.items():
    for word in word_list:
        keyword_count[word] +=1


In [9]:
x,y = keyword_count.keys(),keyword_count.values()
keyword_count_df = pd.DataFrame({"keyword":x,"count":y})
keyword_count_df = keyword_count_df.sort_values(by='count',ascending=False)
negative_keyword_count_df = pd.DataFrame({"keyword":negative_keyword_counts.keys(),"negative_count":negative_keyword_counts.values()})


NameError: name 'negative_keyword_counts' is not defined

In [10]:
all_count = pd.merge(keyword_count_df,negative_keyword_count_df,on="keyword",how="left")
all_count = all_count.fillna(value=0)
all_count = all_count.sort_values(by=['negative_count','count'],ascending=False)


NameError: name 'negative_keyword_count_df' is not defined

In [41]:
input_dir = "/root/autodl-tmp/epsave/data/binary"
positive_file = os.path.join(input_dir,"PositiveData.txt")

positive_df = read_data_file(positive_file,False,5000,head = head)
positive_count_dict = positive_df["keyword"].value_counts().to_dict()

positive_keyword_count_dict = defaultdict(int)
for key,count_num in positive_count_dict.items():
    for word in keyword_dict[key]:
        positive_keyword_count_dict[word] += count_num

positive_count_df = pd.DataFrame({"keyword":positive_keyword_count_dict.keys(),"positive_count":positive_keyword_count_dict.values()})
all_count = pd.merge(all_count,positive_count_df,on="keyword",how="left")
all_count = all_count.fillna(value=0)
all_count = all_count.sort_values(by=['negative_count','positive_count'],ascending=False)


In [42]:
exist_all_count = all_count.query("negative_count>100 & positive_count>100")
exist_all_count["positive_rate"] = exist_all_count["positive_count"] / exist_all_count["negative_count"]
exist_all_count = exist_all_count.sort_values(by=['positive_count','negative_count'],ascending=False)

/tmp/ipykernel_16549/3721869299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exist_all_count["positive_rate"] = exist_all_count["positive_count"] / exist_all_count["negative_count"]


In [43]:
input_dir = "/root/autodl-tmp/epsave/data/binary"
unlabel_file = os.path.join(input_dir,"unlabeldata.txt")

unlabel_df = read_data_file(unlabel_file,False,5000,head = ["Query","keyword"])
unlabel_df_count_dict = unlabel_df["keyword"].value_counts().to_dict()
unlabel_count_df = pd.DataFrame({"keyword":unlabel_df_count_dict.keys(),"unlabel_count":unlabel_df_count_dict.values()})
unlalbe_all_count = pd.merge(exist_all_count,unlabel_count_df,on="keyword",how="left")
unlalbe_all_count = unlalbe_all_count.fillna(value=0)
unlalbe_all_count = unlalbe_all_count.sort_values(by='positive_count',ascending=False)


In [44]:
exist_all_count_unique = exist_all_count.drop_duplicates(['positive_count'],keep='first')
exist_all_count_unique.shape

(47, 5)

In [45]:
max_filter_num = 47
words_by_unlabel = unlalbe_all_count.query("unlabel_count>100").iloc[:max_filter_num,0].values.tolist()
words_by_negative = exist_all_count_unique.iloc[:max_filter_num,0].values.tolist()

In [46]:
def cover_positive_rate(words):
    keyword_set = set()
    for word in words:
        for keyword in euphemism_answer[word]:
            keyword_set.add(keyword)
    cnt = 0
    for keyword in keyword_set:
        if keyword in positive_count_dict.keys():
            cnt += positive_count_dict[keyword]
    print(cnt)
    positive_num = positive_df.shape[0]
    print(cnt/positive_num)


In [47]:
cover_positive_rate(words_by_unlabel)

4332
0.8617465685299384


In [48]:
cover_positive_rate(words_by_negative)


4931
0.9809031231350707


In [49]:
with open("/root/autodl-tmp/epsave/data/binary/classifyword.txt","w") as write_object:
        for word in words_by_negative:
            write_object.write(word+"\n")
